# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 27 2022 9:37AM,249498,19,8699512,"Emersa Waterbox, LLC",Released
1,Oct 27 2022 9:36AM,249495,10,CTF0011473,"Citieffe, Inc.",Released
2,Oct 27 2022 9:36AM,249495,10,CTF0011474,"Citieffe, Inc.",Released
3,Oct 27 2022 9:36AM,249495,10,CTF0011475,"Citieffe, Inc.",Released
4,Oct 27 2022 9:36AM,249495,10,CTF0011476,"Citieffe, Inc.",Released
5,Oct 27 2022 9:33AM,249497,12,U1084,"Uniderm USA, Inc",Released
6,Oct 27 2022 9:33AM,249497,12,U1087,"Uniderm USA, Inc",Released
7,Oct 27 2022 9:33AM,249497,12,U1090,"Uniderm USA, Inc",Released
8,Oct 27 2022 9:33AM,249496,102,7139141,"Snap Medical Industries, LLC",Executing
9,Oct 27 2022 9:08AM,249492,12,SBSB20221027,SugarBear,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,249492,Released,1
18,249495,Released,4
19,249496,Executing,1
20,249497,Released,3
21,249498,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
249492,NaN,1.0
249495,NaN,4.0
249496,1.0,NaN
249497,NaN,3.0
249498,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
248349,0.0,1.0
248599,0.0,1.0
249407,0.0,1.0
249418,0.0,1.0
249447,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
248349,0,1
248599,0,1
249407,0,1
249418,0,1
249447,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,248349,0,1
1,248599,0,1
2,249407,0,1
3,249418,0,1
4,249447,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,248349,,1
1,248599,,1
2,249407,,1
3,249418,,1
4,249447,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 27 2022 9:37AM,249498,19,"Emersa Waterbox, LLC"
1,Oct 27 2022 9:36AM,249495,10,"Citieffe, Inc."
5,Oct 27 2022 9:33AM,249497,12,"Uniderm USA, Inc"
8,Oct 27 2022 9:33AM,249496,102,"Snap Medical Industries, LLC"
9,Oct 27 2022 9:08AM,249492,12,SugarBear
10,Oct 27 2022 8:58AM,249490,10,ISDIN Corporation
57,Oct 27 2022 8:43AM,249488,10,ISDIN Corporation
71,Oct 27 2022 8:35AM,249487,19,"Innogenix, LLC"
72,Oct 27 2022 8:34AM,249485,20,Reliable 1 Laboratories LLC
73,Oct 27 2022 8:21AM,249479,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 27 2022 9:37AM,249498,19,"Emersa Waterbox, LLC",,1
1,Oct 27 2022 9:36AM,249495,10,"Citieffe, Inc.",,4
2,Oct 27 2022 9:33AM,249497,12,"Uniderm USA, Inc",,3
3,Oct 27 2022 9:33AM,249496,102,"Snap Medical Industries, LLC",1,
4,Oct 27 2022 9:08AM,249492,12,SugarBear,,1
5,Oct 27 2022 8:58AM,249490,10,ISDIN Corporation,,47
6,Oct 27 2022 8:43AM,249488,10,ISDIN Corporation,7,7
7,Oct 27 2022 8:35AM,249487,19,"Innogenix, LLC",,1
8,Oct 27 2022 8:34AM,249485,20,Reliable 1 Laboratories LLC,1,
9,Oct 27 2022 8:21AM,249479,16,Sartorius Bioprocess Solutions,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 27 2022 9:37AM,249498,19,"Emersa Waterbox, LLC",1,
1,Oct 27 2022 9:36AM,249495,10,"Citieffe, Inc.",4,
2,Oct 27 2022 9:33AM,249497,12,"Uniderm USA, Inc",3,
3,Oct 27 2022 9:33AM,249496,102,"Snap Medical Industries, LLC",,1
4,Oct 27 2022 9:08AM,249492,12,SugarBear,1,
5,Oct 27 2022 8:58AM,249490,10,ISDIN Corporation,47,
6,Oct 27 2022 8:43AM,249488,10,ISDIN Corporation,7,7
7,Oct 27 2022 8:35AM,249487,19,"Innogenix, LLC",1,
8,Oct 27 2022 8:34AM,249485,20,Reliable 1 Laboratories LLC,,1
9,Oct 27 2022 8:21AM,249479,16,Sartorius Bioprocess Solutions,,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 27 2022 9:37AM,249498,19,"Emersa Waterbox, LLC",1,
1,Oct 27 2022 9:36AM,249495,10,"Citieffe, Inc.",4,
2,Oct 27 2022 9:33AM,249497,12,"Uniderm USA, Inc",3,
3,Oct 27 2022 9:33AM,249496,102,"Snap Medical Industries, LLC",,1
4,Oct 27 2022 9:08AM,249492,12,SugarBear,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 27 2022 9:37AM,249498,19,"Emersa Waterbox, LLC",1.0,NaN
1,Oct 27 2022 9:36AM,249495,10,"Citieffe, Inc.",4.0,NaN
2,Oct 27 2022 9:33AM,249497,12,"Uniderm USA, Inc",3.0,NaN
3,Oct 27 2022 9:33AM,249496,102,"Snap Medical Industries, LLC",NaN,1.0
4,Oct 27 2022 9:08AM,249492,12,SugarBear,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 27 2022 9:37AM,249498,19,"Emersa Waterbox, LLC",1.0,0.0
1,Oct 27 2022 9:36AM,249495,10,"Citieffe, Inc.",4.0,0.0
2,Oct 27 2022 9:33AM,249497,12,"Uniderm USA, Inc",3.0,0.0
3,Oct 27 2022 9:33AM,249496,102,"Snap Medical Industries, LLC",0.0,1.0
4,Oct 27 2022 9:08AM,249492,12,SugarBear,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,997935,63.0,7.0
102,249496,0.0,1.0
12,997866,6.0,0.0
16,249479,0.0,1.0
19,1247374,23.0,17.0
20,249485,0.0,1.0
21,995806,4.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,997935,63.0,7.0
1,102,249496,0.0,1.0
2,12,997866,6.0,0.0
3,16,249479,0.0,1.0
4,19,1247374,23.0,17.0
5,20,249485,0.0,1.0
6,21,995806,4.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,63.0,7.0
1,102,0.0,1.0
2,12,6.0,0.0
3,16,0.0,1.0
4,19,23.0,17.0
5,20,0.0,1.0
6,21,4.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,63.0
1,102,Released,0.0
2,12,Released,6.0
3,16,Released,0.0
4,19,Released,23.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,16,19,20,21
Status,,,,,,,
Executing,7.0,1.0,0.0,1.0,17.0,1.0,0.0
Released,63.0,0.0,6.0,0.0,23.0,0.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,16,19,20,21
0,Executing,7.0,1.0,0.0,1.0,17.0,1.0,0.0
1,Released,63.0,0.0,6.0,0.0,23.0,0.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,16,19,20,21
0,Executing,7.0,1.0,0.0,1.0,17.0,1.0,0.0
1,Released,63.0,0.0,6.0,0.0,23.0,0.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()